In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score
from scipy.stats import ks_2samp
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
%matplotlib inline

C:\Users\jvsn1\Anaconda3\envs\python3_ri\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_train_res = pd.read_csv('database/X_train_res.csv').drop(columns=['Unnamed: 0'])
y_train_res = pd.read_csv('database/y_train_res.csv').drop(columns=['Unnamed: 0'])
X_val = pd.read_csv('database/X_val.csv').drop(columns=['INDEX'])
y_val = pd.read_csv('database/y_val.csv', names=['INDEX', 'IND_BOM_1_1']).drop(columns=['INDEX'])
X_test = pd.read_csv('database/X_test.csv').drop(columns=['INDEX'])
y_test = pd.read_csv('database/y_test.csv', names=['INDEX', 'IND_BOM_1_1']).drop(columns=['INDEX'])

<h2>Variáveis e Métodos Auxiliares</h2>

In [3]:
input_dim = len(list(X_train_res)) # Dimensão de entrada do dataset

In [67]:
'''
    * Método para calcular a distribuição acumulada das classes, de 0.0 a 1.0
    * Utilizado para o cálculo do KS
'''
def calc_distr(y_true:pd.DataFrame, y_pred_proba:np.array):
    ac_distr_0 = np.zeros(101)
    ac_distr_1 = np.zeros(101)
    count_classes = y_true['IND_BOM_1_1'].value_counts()
    for i in range(1, 101):
        lim = i/100.0
        ac_classes = y_true[y_pred_proba <= lim]['IND_BOM_1_1'].value_counts()
        ac_distr_0[i] += ac_classes.get(0, 0) 
        ac_distr_1[i] += ac_classes.get(1, 0)
    return (ac_distr_0/count_classes[0], ac_distr_1/count_classes[1])

In [68]:
'''
    * Método para calcular as métricas a serem impressas
'''

def calc_metrics(y_true, y_pred, y_pred_proba):
    cm = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    metrics = {
        'cm': cm,
        'precision': precision,
        'accuracy': accuracy,
        'f1': f1,
        'recall': recall
    }
    if not y_pred_proba is None:
        ac_distr0, ac_distr1 = calc_distr(y_true, y_pred_proba)
        ks = ks_2samp(ac_distr0, ac_distr1)
        auroc = roc_auc_score(y_true, y_pred_proba)
        aps = average_precision_score(y_true, y_pred_proba)
        metrics['ks'] = ks[0]
        metrics['auroc'] = auroc
        metrics['aps'] = aps
    return metrics

In [95]:
'''
    * Método para imprimir todas as métricas
'''
def print_metrics(metrics:dict):
    print('Matriz de Confusão:', end='\n\n')
    print(pd.DataFrame(metrics['cm'], columns=['T', 'F'], index=['T', 'F']), end='\n\n')
    print('Área Sob Curva ROC: %.5f'%(metrics['auroc']), end = '\n\n')
    print('KS-Score: %.5f'%(metrics['ks']), end='\n\n')
    print('Precisão Média de Previsão: %.5f'%(metrics['aps']), end='\n\n')
    print('Precisão: %.5f'%(metrics['precision']), end='\n\n')
    print('Acurácia: %.5f'%(metrics['accuracy']), end='\n\n')
    print('Recall: %.5f'%(metrics['recall']), end='\n\n')
    print('F1-Score: %.5f'%(metrics['f1']), end='\n\n')

<h2>Treinando MLPs</h2>
<p>Para o treino de MLPs foram testadas diversas formas de otimizar hiperparâmetros como: learning rate, algoritmo de otimização, número de camadas escondidas, etc. A estimativa do learning rate e do número de camadas escondidas foi feita automaticamente a partir do conjunto de validação.</p>

<h3>O MLP Básico</h3>
<p>O MLP é montado com base na biblioteca Keras e segue uma estrutura básica:</p>
<ul>
    <li>O MLP em si é uma classe Sequential e nela serão adicionadas as camadas de entrada, saída e internas.</li>
    <li>As camadas são uma classe Dense. Passamos para o construtor dessa classe o número de nodes, a função de ativação e, no caso de ser a primeira camada, a dimensão da entrada.</li>
    <li>Antes de treinar o MLP, devemos compilar o mesmo e setar parâmetros como a função de otimização, algumas métricas opcionais como 'accuracy' e a função de perda, que retorna uma métrica para avaliarmos o MLP</li>
    <li>Para treinarmos o modelo utilizaremos a função fit que recebe como parâmetros X sendo as amostras para treinamento, y sendo as classes de cada amostra, batch_size sendo o comprimento do batch utilizado no treinamento, o número de epochs sendo quantas vezes iremos dar entrada com o dataset para o treinamento, validation_data sendo os dados utilizados para validação e callbacks para pararmos o treinamento com certas condições.</li>
</ul>
<p>Para a primeira fase, procuraremos funções de ativação, número de nós na camada de entrada e função de otimização que melhorem o acerto no conjunto de validação</p>

<h3>Estimando Hiperparâmetros</h3>
<ul>
    <li>Função de Ativação</li>
    <li>Função de Otimização</li>
    <li>Camadas escondidas</li>
</ul>

<h4>Número de Neurônios na Camada de Entrada</h4>
<p>Incialmente verificamos se o número de neurônios na camada de entrada influencia muito na qualidade do MLP</p>

<h5>20 Neurônios</h5>

In [93]:
mlp = Sequential()
mlp.add(Dense(20, activation='tanh', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val));
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 9s 29us/step - loss: 0.6387 - acc: 0.6346 - val_loss: 0.6335 - val_acc: 0.6313
Epoch 2/100000
307646/307646 [==============================] - 9s 29us/step - loss: 0.6231 - acc: 0.6514 - val_loss: 0.6689 - val_acc: 0.5912
Epoch 3/100000
307646/307646 [==============================] - 9s 28us/step - loss: 0.6143 - acc: 0.6597 - val_loss: 0.6380 - val_acc: 0.6244
Epoch 4/100000
307646/307646 [==============================] - 9s 29us/step - loss: 0.6057 - acc: 0.6688 - val_loss: 0.6385 - val_acc: 0.6253


In [94]:
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Matriz de Confusão:

       T      F
T  28819  15433
F  31997  52186

Área Sob Curva ROC: 0.68849

KS-Score: 0.16832

Precisão Média de Previsão: 0.79984

Precisão: 0.77177

Acurácia: 0.63071

Recall: 0.61991

F1-Score: 0.68755



<h5>40 Neurônios</h5>

In [99]:
mlp = Sequential()
mlp.add(Dense(40, activation='tanh', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6369 - acc: 0.6379 - val_loss: 0.6550 - val_acc: 0.6046
Epoch 2/100000
307646/307646 [==============================] - 9s 29us/step - loss: 0.6201 - acc: 0.6548 - val_loss: 0.6331 - val_acc: 0.6281
Epoch 3/100000
307646/307646 [==============================] - 10s 34us/step - loss: 0.6097 - acc: 0.6657 - val_loss: 0.6417 - val_acc: 0.6207
Epoch 4/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.5991 - acc: 0.6751 - val_loss: 0.6366 - val_acc: 0.6317
Epoch 5/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.5875 - acc: 0.6839 - val_loss: 0.6165 - val_acc: 0.6538
Epoch 6/100000
307646/307646 [==============================] - 10s 31us/step - loss: 0.5757 - acc: 0.6926 - val_loss: 0.6332 - val_acc: 0.6391
Epoch 7/100000
307646/307646 [==============================] - 10s 31us/step - loss: 

In [100]:
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Matriz de Confusão:

       T      F
T  27008  17244
F  30156  54027

Área Sob Curva ROC: 0.67645

KS-Score: 0.17822

Precisão Média de Previsão: 0.79120

Precisão: 0.75805

Acurácia: 0.63094

Recall: 0.64178

F1-Score: 0.69509



<h5>80 Neurônios</h5>

In [101]:
mlp = Sequential()
mlp.add(Dense(80, activation='tanh', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.6355 - acc: 0.6380 - val_loss: 0.6120 - val_acc: 0.6508
Epoch 2/100000
307646/307646 [==============================] - 11s 34us/step - loss: 0.6180 - acc: 0.6563 - val_loss: 0.6181 - val_acc: 0.6480
Epoch 3/100000
307646/307646 [==============================] - 10s 34us/step - loss: 0.6053 - acc: 0.6696 - val_loss: 0.6299 - val_acc: 0.6390
Epoch 4/100000
307646/307646 [==============================] - 10s 34us/step - loss: 0.5898 - acc: 0.6834 - val_loss: 0.6581 - val_acc: 0.6121


In [102]:
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Matriz de Confusão:

       T      F
T  30713  13539
F  35855  48328

Área Sob Curva ROC: 0.68321

KS-Score: 0.16832

Precisão Média de Previsão: 0.79632

Precisão: 0.78116

Acurácia: 0.61542

Recall: 0.57408

F1-Score: 0.66180



<p>Analisando principalmente o KS, vemos que o melhor valor foi encontrado entre 20, 40 e 80 foi 40.</p>

<h4>Função de Ativação na Camada de Entrada</h4>
<p>Para problemas de classificação, as melhores funções de ativação são relu, sigmoid e tanh. Testaremos as 3 na camada de entrada</p>

In [105]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 11s 36us/step - loss: 0.6415 - acc: 0.6330 - val_loss: 0.6468 - val_acc: 0.6175
Epoch 2/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6288 - acc: 0.6459 - val_loss: 0.6457 - val_acc: 0.6172
Epoch 3/100000
307646/307646 [==============================] - 11s 36us/step - loss: 0.6197 - acc: 0.6554 - val_loss: 0.6231 - val_acc: 0.6424
Epoch 4/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6120 - acc: 0.6629 - val_loss: 0.6573 - val_acc: 0.6046
Epoch 5/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.6032 - acc: 0.6713 - val_loss: 0.6338 - val_acc: 0.6275
Epoch 6/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.5940 - acc: 0.6796 - val_loss: 0.6350 - val_acc: 0.6302
Matriz de Confusão:

       T      F
T  28250  16002
F  31085  53098

Área Sob Curva 

In [106]:
mlp = Sequential()
mlp.add(Dense(40, activation='relu', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6334 - acc: 0.6413 - val_loss: 0.6435 - val_acc: 0.6146
Epoch 2/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6155 - acc: 0.6599 - val_loss: 0.6262 - val_acc: 0.6396
Epoch 3/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.6046 - acc: 0.6710 - val_loss: 0.6155 - val_acc: 0.6544
Epoch 4/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.5935 - acc: 0.6811 - val_loss: 0.6280 - val_acc: 0.6417
Epoch 5/100000
307646/307646 [==============================] - 11s 34us/step - loss: 0.5813 - acc: 0.6900 - val_loss: 0.6230 - val_acc: 0.6481
Epoch 6/100000
307646/307646 [==============================] - 10s 34us/step - loss: 0.5693 - acc: 0.6974 - val_loss: 0.6278 - val_acc: 0.6436
Matriz de Confusão:

       T      F
T  23779  20473
F  24781  59402

Área Sob Curva 

<p>Modificando as funções de ativação na camada de entrada também não variou muito a perda média. Testamos então na camada de saída.</p>

In [109]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='relu')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 12s 38us/step - loss: 0.6548 - acc: 0.6216 - val_loss: 0.6407 - val_acc: 0.6270
Epoch 2/100000
307646/307646 [==============================] - 10s 34us/step - loss: 0.6361 - acc: 0.6396 - val_loss: 0.6450 - val_acc: 0.6157
Epoch 3/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6277 - acc: 0.6478 - val_loss: 0.6353 - val_acc: 0.6309
Epoch 4/100000
307646/307646 [==============================] - 12s 37us/step - loss: 0.6207 - acc: 0.6547 - val_loss: 0.6765 - val_acc: 0.5865
Epoch 5/100000
307646/307646 [==============================] - 12s 37us/step - loss: 0.6211 - acc: 0.6576 - val_loss: 0.6312 - val_acc: 0.6332
Epoch 6/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.6126 - acc: 0.6636 - val_loss: 0.6460 - val_acc: 0.6197
Epoch 7/100000
307646/307646 [==============================] - 10s 32us/step - loss:

In [110]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='tanh')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 11s 37us/step - loss: 0.6469 - acc: 0.6278 - val_loss: 0.6213 - val_acc: 0.6521
Epoch 2/100000
307646/307646 [==============================] - 11s 36us/step - loss: 0.6326 - acc: 0.6435 - val_loss: 0.6249 - val_acc: 0.6352
Epoch 3/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6201 - acc: 0.6552 - val_loss: 0.6700 - val_acc: 0.5987
Epoch 4/100000
307646/307646 [==============================] - 11s 36us/step - loss: 0.6084 - acc: 0.6661 - val_loss: 0.6036 - val_acc: 0.6640
Epoch 5/100000
307646/307646 [==============================] - 11s 36us/step - loss: 0.6007 - acc: 0.6730 - val_loss: 0.6454 - val_acc: 0.6307
Epoch 6/100000
307646/307646 [==============================] - 11s 37us/step - loss: 0.5907 - acc: 0.6777 - val_loss: 0.6273 - val_acc: 0.6432
Epoch 7/100000
307646/307646 [==============================] - 11s 34us/step - loss:

<p>Os melhores valores de KS foram encontrados quando utilizando sigmoid em ambas as camadas.</p>

<h4>Algoritmo de Otimização</h4>

In [112]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 10s 34us/step - loss: 0.6426 - acc: 0.6317 - val_loss: 0.6671 - val_acc: 0.5993
Epoch 2/100000
307646/307646 [==============================] - 10s 31us/step - loss: 0.6338 - acc: 0.6411 - val_loss: 0.6487 - val_acc: 0.6130
Epoch 3/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6261 - acc: 0.6489 - val_loss: 0.6361 - val_acc: 0.6243
Epoch 4/100000
307646/307646 [==============================] - 9s 31us/step - loss: 0.6206 - acc: 0.6537 - val_loss: 0.6452 - val_acc: 0.6168
Epoch 5/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6158 - acc: 0.6582 - val_loss: 0.6328 - val_acc: 0.6317
Epoch 6/100000
307646/307646 [==============================] - 11s 36us/step - loss: 0.6104 - acc: 0.6651 - val_loss: 0.6223 - val_acc: 0.6439
Epoch 7/100000
307646/307646 [==============================] - 10s 32us/step - loss: 

In [113]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.6441 - acc: 0.6289 - val_loss: 0.6761 - val_acc: 0.5868
Epoch 2/100000
307646/307646 [==============================] - 11s 37us/step - loss: 0.6361 - acc: 0.6388 - val_loss: 0.6389 - val_acc: 0.6240
Epoch 3/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.6294 - acc: 0.6454 - val_loss: 0.6431 - val_acc: 0.6180
Epoch 4/100000
307646/307646 [==============================] - 11s 37us/step - loss: 0.6229 - acc: 0.6510 - val_loss: 0.6224 - val_acc: 0.6392
Epoch 5/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6176 - acc: 0.6568 - val_loss: 0.6476 - val_acc: 0.6163
Epoch 6/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6123 - acc: 0.6623 - val_loss: 0.6334 - val_acc: 0.6315
Epoch 7/100000
307646/307646 [==============================] - 10s 34us/step - loss:

In [114]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.6407 - acc: 0.6328 - val_loss: 0.6228 - val_acc: 0.6411
Epoch 2/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6271 - acc: 0.6470 - val_loss: 0.6155 - val_acc: 0.6480
Epoch 3/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6198 - acc: 0.6548 - val_loss: 0.6355 - val_acc: 0.6251
Epoch 4/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6126 - acc: 0.6627 - val_loss: 0.6326 - val_acc: 0.6316
Epoch 5/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6034 - acc: 0.6713 - val_loss: 0.6552 - val_acc: 0.6106
Matriz de Confusão:

       T      F
T  31339  12913
F  36753  47430

Área Sob Curva ROC: 0.68871

KS-Score: 0.17822

Precisão Média de Previsão: 0.80183

Precisão: 0.78601

Acurácia: 0.61330

Recall: 0.56342

F1-Score: 0.65635



In [115]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 12s 38us/step - loss: 0.6752 - acc: 0.5822 - val_loss: 0.6623 - val_acc: 0.6026
Epoch 2/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6548 - acc: 0.6202 - val_loss: 0.6635 - val_acc: 0.5985
Epoch 3/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6478 - acc: 0.6265 - val_loss: 0.6440 - val_acc: 0.6219
Epoch 4/100000
307646/307646 [==============================] - 11s 34us/step - loss: 0.6444 - acc: 0.6297 - val_loss: 0.6501 - val_acc: 0.6134
Epoch 5/100000
307646/307646 [==============================] - 11s 37us/step - loss: 0.6422 - acc: 0.6323 - val_loss: 0.6505 - val_acc: 0.6126
Epoch 6/100000
307646/307646 [==============================] - 10s 34us/step - loss: 0.6407 - acc: 0.6342 - val_loss: 0.6384 - val_acc: 0.6256
Epoch 7/100000
307646/307646 [==============================] - 11s 35us/step - loss:

<p>Ambos algoritmos adamax e adadelta geraram ks semelhante, porém o adamax possui uma melhor aucroc, porém o adam gerou o melhor ks. SGD foi o pior algoritmo de otimização.</p>

<h4>Camadas Escondidas</h4>

In [117]:
hidden_nodes = 30

In [118]:
for hidden_layers in range(6):
    mlp = Sequential()
    mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
    for i in range(hidden_layers):
        mlp.add(Dense(hidden_nodes, activation='relu', input_dim=input_dim))
    mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
    mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
            callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
    y_pred = mlp.predict_classes(X_test)
    y_pred_proba = mlp.predict(X_test)
    metrics = calc_metrics(y_test, y_pred, y_pred_proba)
    print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.6422 - acc: 0.6319 - val_loss: 0.6265 - val_acc: 0.6387
Epoch 2/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.6293 - acc: 0.6449 - val_loss: 0.6188 - val_acc: 0.6434
Epoch 3/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.6204 - acc: 0.6542 - val_loss: 0.6351 - val_acc: 0.6276
Epoch 4/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6130 - acc: 0.6620 - val_loss: 0.6297 - val_acc: 0.6345
Epoch 5/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.6044 - acc: 0.6699 - val_loss: 0.6377 - val_acc: 0.6280
Matriz de Confusão:

       T      F
T  29134  15118
F  32139  52044

Área Sob Curva ROC: 0.68977

KS-Score: 0.16832

Precisão Média de Previsão: 0.80174

Precisão: 0.77490

Acurácia: 0.63206

Recall: 0.61822

F1-Score: 0.68775



<p>O maior KS foi obtido com 5 camadas escondidas, cada uma com 30 neurônios</p>

In [122]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
for i in range(5):
    mlp.add(Dense(hidden_nodes, activation='relu', input_dim=input_dim))
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 18s 59us/step - loss: 0.6386 - acc: 0.6344 - val_loss: 0.6257 - val_acc: 0.6315
Epoch 2/100000
307646/307646 [==============================] - 17s 55us/step - loss: 0.6214 - acc: 0.6534 - val_loss: 0.6287 - val_acc: 0.6320
Epoch 3/100000
307646/307646 [==============================] - 16s 51us/step - loss: 0.6115 - acc: 0.6635 - val_loss: 0.6265 - val_acc: 0.6358
Epoch 4/100000
307646/307646 [==============================] - 16s 51us/step - loss: 0.6024 - acc: 0.6729 - val_loss: 0.6241 - val_acc: 0.6428
Epoch 5/100000
307646/307646 [==============================] - 16s 51us/step - loss: 0.5917 - acc: 0.6816 - val_loss: 0.6138 - val_acc: 0.6561
Epoch 6/100000
307646/307646 [==============================] - 16s 51us/step - loss: 0.5796 - acc: 0.6893 - val_loss: 0.6357 - val_acc: 0.6437
Epoch 7/100000
307646/307646 [==============================] - 16s 51us/step - loss:

In [123]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
for i in range(5):
    mlp.add(Dense(hidden_nodes, activation='relu', input_dim=input_dim))
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 17s 56us/step - loss: 0.1074 - acc: 0.6227 - val_loss: 0.1077 - val_acc: 0.5624
Epoch 2/100000
307646/307646 [==============================] - 18s 58us/step - loss: 0.1039 - acc: 0.6418 - val_loss: 0.1045 - val_acc: 0.5944
Epoch 3/100000
307646/307646 [==============================] - 17s 57us/step - loss: 0.1019 - acc: 0.6529 - val_loss: 0.1073 - val_acc: 0.5827
Epoch 4/100000
307646/307646 [==============================] - 16s 52us/step - loss: 0.0993 - acc: 0.6641 - val_loss: 0.1063 - val_acc: 0.5944
Epoch 5/100000
307646/307646 [==============================] - 17s 55us/step - loss: 0.0968 - acc: 0.6712 - val_loss: 0.1056 - val_acc: 0.6108
Matriz de Confusão:

       T      F
T  30939  13313
F  36229  47954

Área Sob Curva ROC: 0.68923

KS-Score: 0.20792

Precisão Média de Previsão: 0.80262

Precisão: 0.78271

Acurácia: 0.61426

Recall: 0.56964

F1-Score: 0.65939



In [127]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
for i in range(5):
    mlp.add(Dense(hidden_nodes, activation='relu', input_dim=input_dim))
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 20s 65us/step - loss: 0.1072 - acc: 0.6233 - val_loss: 0.1056 - val_acc: 0.5686
Epoch 2/100000
307646/307646 [==============================] - 17s 55us/step - loss: 0.1039 - acc: 0.6415 - val_loss: 0.1050 - val_acc: 0.5929
Epoch 3/100000
307646/307646 [==============================] - 17s 55us/step - loss: 0.1022 - acc: 0.6501 - val_loss: 0.1062 - val_acc: 0.5956
Epoch 4/100000
307646/307646 [==============================] - 17s 55us/step - loss: 0.1005 - acc: 0.6592 - val_loss: 0.1044 - val_acc: 0.6028
Epoch 5/100000
307646/307646 [==============================] - 17s 55us/step - loss: 0.0985 - acc: 0.6675 - val_loss: 0.1039 - val_acc: 0.5990
Epoch 6/100000
307646/307646 [==============================] - 17s 55us/step - loss: 0.0960 - acc: 0.6770 - val_loss: 0.1064 - val_acc: 0.6107
Epoch 7/100000
307646/307646 [==============================] - 17s 56us/step - loss:

<h4>Batch Size</h4>

In [131]:
mlp = Sequential()
mlp.add(Dense(40, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
for i in range(5):
    mlp.add(Dense(hidden_nodes, activation='relu', input_dim=input_dim))
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=32, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)
metrics = calc_metrics(y_test, y_pred, y_pred_proba)
print_metrics(metrics)

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 32s 104us/step - loss: 0.1072 - acc: 0.6230 - val_loss: 0.1094 - val_acc: 0.5573
Epoch 2/100000
307646/307646 [==============================] - 29s 93us/step - loss: 0.1035 - acc: 0.6433 - val_loss: 0.1093 - val_acc: 0.5700
Epoch 3/100000
307646/307646 [==============================] - 28s 90us/step - loss: 0.1009 - acc: 0.6559 - val_loss: 0.1120 - val_acc: 0.5802
Epoch 4/100000
307646/307646 [==============================] - 28s 90us/step - loss: 0.0982 - acc: 0.6673 - val_loss: 0.1060 - val_acc: 0.6174
Epoch 5/100000
307646/307646 [==============================] - 28s 89us/step - loss: 0.0958 - acc: 0.6779 - val_loss: 0.1039 - val_acc: 0.6208
Epoch 6/100000
307646/307646 [==============================] - 28s 92us/step - loss: 0.0938 - acc: 0.6842 - val_loss: 0.1028 - val_acc: 0.6156
Epoch 7/100000
307646/307646 [==============================] - 28s 90us/step - loss

In [6]:
opt_hidden_layer = 3
mlp = Sequential()
mlp.add(Dense(30, activation='relu', input_dim=input_dim)) # Camada de entrada
for i in range(opt_hidden_layer):
    mlp.add(Dense(hidden_nodes, activation='relu')) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=32, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
print('Perda Média: %.2f'%(np.mean(history.history['val_loss'])))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 17s 54us/step - loss: 0.6336 - acc: 0.6397 - val_loss: 0.6490 - val_acc: 0.6185
Epoch 2/100000
307646/307646 [==============================] - 13s 44us/step - loss: 0.6129 - acc: 0.6631 - val_loss: 0.6194 - val_acc: 0.6510
Epoch 3/100000
307646/307646 [==============================] - 13s 44us/step - loss: 0.5956 - acc: 0.6755 - val_loss: 0.6396 - val_acc: 0.6322
Epoch 4/100000
307646/307646 [==============================] - 14s 45us/step - loss: 0.5794 - acc: 0.6849 - val_loss: 0.6212 - val_acc: 0.6556
Epoch 5/100000
307646/307646 [==============================] - 14s 44us/step - loss: 0.5684 - acc: 0.6916 - val_loss: 0.6170 - val_acc: 0.6552
Epoch 6/100000
307646/307646 [==============================] - 14s 46us/step - loss: 0.5604 - acc: 0.6960 - val_loss: 0.6316 - val_acc: 0.6452
Epoch 7/100000
307646/307646 [==============================] - 14s 44us/step - loss:

<p>Para o tamanho do batch = 32 obtivemos a menor perda.</p>

In [ ]:
test_evaluation = mlp.evaluate(X_test, y_test, batch_size=32)
print("Conjunto de Teste:\n\tPerda: %.3f\n\tAcurácia: %.3f"%(test_evaluation[0],test_evaluation[1]))